# 一、使用Transformer和PyTorch构建日中翻译模型的教程
使用Jupyter Notebook、PyTorch、Torchtext和SentencePiece。

## 1.1  配置cuda环境
配置教程参考：https://zhuanlan.zhihu.com/p/679245051  
https://blog.csdn.net/AI_dataloads/article/details/133070649  
  
配置好cuda和cudnn后，在命令行输入nvcc-V，显示如下内容：  
  
![](https://p.sda1.dev/18/eeecaf54b7ead78b52f7da8a8613bea0/1.png).  

我安装的cuda版本是v12.1

然后安装对应cuda版本的pytorch
![](https://p.sda1.dev/18/8e2897461591f460269d872ee89bd6de/2.png)

## 1.2 导入所需的包
首先，让我们确保我们的系统中已安装以下包，如果发现有一些包缺失，请确保安装它们。


In [ ]:
import math
import torchtext
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from collections import Counter
from torchtext.vocab import Vocab
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer
import io
import time
import pandas as pd
import numpy as np
import pickle
import tqdm
import sentencepiece as spm

## 1.3 查看实验环境
我的运行平台是自己的笔记本Legion R9000P ARX8  
GPU是RTX 4060 Laptop  
使用的pytorch版本是2.3.1+cu121

In [31]:
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(0)) 

NVIDIA GeForce RTX 4060 Laptop GPU


In [32]:
print(torch.__version__)

2.3.1+cu121


In [33]:
print(device)

cuda


## 1.4 获取平行数据集
 
使用中日数据集zh-ja.bicleaner05.tx，包含803892个中日平行句子
![](https://p.sda1.dev/18/1c12c8dd09537fbc3886230f3035a396/3.png)

In [51]:
df = pd.read_csv('zh-ja.bicleaner05.txt', sep='\\t', engine='python', header=None)
# 将中文数据存储在 trainen 列表中，转换为列表形式
trainen = df[2].values.tolist()#[:10000]

# 将日文数据存储在 trainja 列表中，转换为列表形式
trainja = df[3].values.tolist()#[:10000]

# trainen.pop(5972)
# trainja.pop(5972)

这是数据集中包含的一个句子的示例。



In [52]:
print(trainen[500])
print(trainja[500])

Chinese HS Code Harmonized Code System < HS编码 2905 无环醇及其卤化、磺化、硝化或亚硝化衍生物 HS Code List (Harmonized System Code) for US, UK, EU, China, India, France, Japan, Russia, Germany, Korea, Canada ...
Japanese HS Code Harmonized Code System < HSコード 2905 非環式アルコール並びにそのハロゲン化誘導体、スルホン化誘導体、ニトロ化誘導体及びニトロソ化誘導体 HS Code List (Harmonized System Code) for US, UK, EU, China, India, France, Japan, Russia, Germany, Korea, Canada ...


我们也可以使用不同的平行数据集来配合本文，只需确保我们可以将数据处理成上面所示的两个字符串列表，其中包含日语和中文句子。

## 1.5 准备分词器
与英语或其他字母语言不同，日语句子中不包含空格来分隔单词。我们可以使用 JParaCrawl 提供的分词器，它们使用 SentencePiece 分别针对日语和英语创建而成。您可以访问 JParaCrawl 网站下载它们。

In [5]:
# 使用 spm.SentencePieceProcessor 加载英语的分词器，指定模型文件为 'spm.en.nopretok.model'
en_tokenizer = spm.SentencePieceProcessor(model_file='spm.en.nopretok.model')

# 使用 spm.SentencePieceProcessor 加载日语的分词器，指定模型文件为 'spm.ja.nopretok.model'
ja_tokenizer = spm.SentencePieceProcessor(model_file='spm.ja.nopretok.model')


在加载分词器之后，您可以通过执行以下代码来测试它们。  
使用 encode 方法将输入字符串编码为整数列表。  


In [44]:
encoded_str = en_tokenizer.encode("All residents aged 20 to 59 years who live in Japan must enroll in public pension system.")
print(encoded_str)

[227, 2980, 8863, 373, 8, 9381, 126, 91, 649, 11, 93, 240, 19228, 11, 419, 14926, 102, 5]


In [46]:
encoded_str = ja_tokenizer.encode("年金 日本に住んでいる20歳~60歳の全ての人は、公的年金制度に加入しなければなりません。")
print(encoded_str)

[4, 31, 346, 912, 10050, 222, 1337, 372, 820, 4559, 858, 750, 3, 13118, 31, 346, 2000, 10, 8978, 5461, 5]


## 1.6 构建TorchText的词汇表对象并将句子转换为Torch张量
使用标记器和原始句子，我们接着构建从TorchText导入的Vocab对象。这个过程可能需要几秒钟或几分钟，这取决于我们数据集的大小和计算能力。不同的标记器也会影响构建词汇表所需的时间。

In [48]:
# 定义构建词汇表的函数
def build_vocab(sentences, tokenizer):
  counter = Counter()
  # 遍历句子，统计词频
  for sentence in sentences:
    counter.update(tokenizer.encode(sentence, out_type=str))
  # 使用Counter构建词汇表对象，并指定特殊标记
  return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

# 分别构建日语和中文的词汇表对象
ja_vocab = build_vocab(trainja, ja_tokenizer)
en_vocab = build_vocab(trainen, en_tokenizer)

当我们有了词汇表对象之后，我们可以使用词汇表和标记器对象来为我们的训练数据构建张量。

In [49]:
# 定义数据处理函数
def data_process(ja, en):
  data = []
  # 遍历日语和中文句子
  for (raw_ja, raw_en) in zip(ja, en):
    # 将句子转换为张量并存储在data列表中
    ja_tensor_ = torch.tensor([ja_vocab[token] for token in ja_tokenizer.encode(raw_ja.rstrip("\n"), out_type=str)],
                            dtype=torch.long)
    en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer.encode(raw_en.rstrip("\n"), out_type=str)],
                            dtype=torch.long)
    data.append((ja_tensor_, en_tensor_))
  return data

## 1.7 创建DataLoader对象，在训练过程中进行迭代
在这里，我将BATCH_SIZE设置为8，以防止“cuda内存不足”的情况发生，但这取决于各种因素，比如您的机器内存容量、数据大小等，因此根据您的需求随时可以更改批处理大小（注意：PyTorch的教程使用Multi30k German-English数据集将批处理大小设置为128）。

In [8]:
BATCH_SIZE = 8  # 设置批处理大小为8
PAD_IDX = ja_vocab['<pad>']  # 获取填充标记的索引
BOS_IDX = ja_vocab['<bos>']  # 获取句子开头标记的索引
EOS_IDX = ja_vocab['<eos>']  # 获取句子结尾标记的索引

# 定义生成批数据的函数
def generate_batch(data_batch):
  ja_batch, en_batch = [], []
  # 遍历数据批次
  for (ja_item, en_item) in data_batch:
    # 在日语句子开头和结尾添加起始和结束标记，并存储在ja_batch中
    ja_batch.append(torch.cat([torch.tensor([BOS_IDX]), ja_item, torch.tensor([EOS_IDX])], dim=0))
    # 在中文句子开头和结尾添加起始和结束标记，并存储在en_batch中
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  # 对日语句子进行填充
  ja_batch = pad_sequence(ja_batch, padding_value=PAD_IDX)
  # 对中文句子进行填充
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return ja_batch, en_batch

# 创建用于训练的数据迭代器
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)


## 1.8 序列到序列的Transformer
接下来的一些代码和文本解释（用斜体书写）来自原始的PyTorch教程[https://pytorch.org/tutorials/beginner/translation_transformer.html]。  
除了BATCH_SIZE和单词de_vocab被更改为ja_vocab之外，我没有做任何改动。

Transformer是一种Seq2Seq模型，是在“Attention is all you need”论文中提出的，用于解决机器翻译任务。Transformer模型由一个包含固定层数的编码器块和解码器块组成。

编码器通过一系列的多头注意力和前馈网络层处理输入序列。编码器的输出被称为记忆，与目标张量一起被馈送到解码器中。编码器和解码器通过teacher forcing技术进行端到端的训练。

In [9]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)

class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        
        # 创建编码器层
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        # 创建Transformer编码器
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        
        # 创建解码器层
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        # 创建Transformer解码器
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        # 线性层，用于生成输出词汇
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        
        # 编码器和解码器的token嵌入
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        
        # 位置编码
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        # 计算源语言和目标语言的嵌入并加上位置编码
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        
        # 编码器处理源语言
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        
        # 解码器生成输出
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        
        # 通过线性层生成最终的输出词汇
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        # 编码器处理源语言并返回编码结果
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        # 解码器根据编码结果生成目标语言的输出
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)


文本标记通过使用标记嵌入表示。位置编码被添加到标记嵌入中，引入单词顺序的概念。

In [10]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        """
        初始化PositionalEncoding模块。

        参数:
        emb_size (int): 嵌入大小。
        dropout: 丢弃率。
        maxlen (int): 序列的最大长度。默认为5000。
        """
        super(PositionalEncoding, self).__init__()
        # 计算位置编码
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        """
        PositionalEncoding模块的前向传播。

        参数:
        token_embedding (Tensor): 标记嵌入张量。

        返回:
        Tensor: 添加了位置编码的标记嵌入张量。
        """
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        """
        初始化TokenEmbedding模块。

        参数:
        vocab_size (int): 词汇表的大小。
        emb_size (int): 嵌入大小。
        """
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        """
        TokenEmbedding模块的前向传播。

        参数:
        tokens (Tensor): 输入标记。

        返回:
        Tensor: 标记嵌入。
        """
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


我们创建一个后续词掩码，以阻止目标词与其后续词进行关联，还创建了用于屏蔽源和目标填充标记的掩码。


In [50]:
def generate_square_subsequent_mask(sz):
    """
    生成一个用于Transformer解码器的后续词掩码。

    参数:
    sz (int): 序列长度。

    返回:
    Tensor: 后续词掩码张量。
    """
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    """
    创建用于屏蔽源和目标填充标记的掩码。

    参数:
    src (Tensor): 源序列张量。
    tgt (Tensor): 目标序列张量。

    返回:
    Tensor: 源掩码，目标掩码，源填充标记掩码，目标填充标记掩码。
    """
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


定义模型参数并实例化模型


In [12]:
# 定义源语言词汇表大小
SRC_VOCAB_SIZE = len(ja_vocab)

# 定义目标语言词汇表大小
TGT_VOCAB_SIZE = len(en_vocab)

# 定义词嵌入维度
EMB_SIZE = 512

# 定义注意力头数
NHEAD = 8

# 定义前馈神经网络隐藏层维度
FFN_HID_DIM = 512

# 定义批处理大小
BATCH_SIZE = 16

# 定义编码器层数
NUM_ENCODER_LAYERS = 3

# 定义解码器层数
NUM_DECODER_LAYERS = 3

# 定义训练轮数
NUM_EPOCHS = 16


# 实例化Seq2SeqTransformer模型
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

# 初始化模型参数
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

# 将模型移动到指定设备
transformer = transformer.to(device)

# 定义损失函数
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# 定义优化器
optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

# 定义训练函数
def train_epoch(model, train_iter, optimizer):
    model.train()
    losses = 0
    for idx, (src, tgt) in  enumerate(train_iter):
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                       src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
    return losses / len(train_iter)

# 定义评估函数
def evaluate(model, val_iter):
    model.eval()
    losses = 0
    for idx, (src, tgt) in (enumerate(valid_iter)):
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                       src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(val_iter)


D:\Python3.11\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


## 1.9 开始训练
最终，在准备好必要的类和函数之后，我们准备训练我们的模型。毋庸置疑，完成训练所需的时间可能会因计算能力、参数和数据集大小等因素而大大不同。

当我使用来自JParaCrawl的完整句子列表进行训练时，每种语言大约有590万句子，在这里我使用的是单个NVIDIA GeForce RTX 4060 GPU进行训练


In [13]:
# 对每个轮次进行迭代
for epoch in tqdm.tqdm(range(1, NUM_EPOCHS+1)):
    start_time = time.time()
    
    # 训练一个轮次并计算损失
    train_loss = train_epoch(transformer, train_iter, optimizer)
    
    end_time = time.time()
    
    # 打印轮次、训练损失和轮次时间
    print(f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Epoch time = {(end_time - start_time):.3f}s")

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]D:\Python3.11\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
D:\Python3.11\Lib\site-packages\torch\nn\functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
  6%|█████                                                                           | 1/16 [12:51<3:12:56, 771.74s/it]

Epoch: 1, Train loss: 4.472, Epoch time = 771.740s


 12%|██████████                                                                      | 2/16 [24:37<2:51:04, 733.15s/it]

Epoch: 2, Train loss: 3.484, Epoch time = 706.130s


 19%|███████████████                                                                 | 3/16 [36:15<2:35:22, 717.09s/it]

...

Epoch: 14, Train loss: 1.843, Epoch time = 666.342s


 94%|██████████████████████████████████████████████████████████████████████████     | 15/16 [2:51:53<11:11, 671.74s/it]

Epoch: 15, Train loss: 1.805, Epoch time = 665.383s


100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [3:02:57<00:00, 686.06s/it]

Epoch: 16, Train loss: 1.769, Epoch time = 663.496s


   
整个训练总共持续3个小时，GPU满载。我尝试过在kaggle平台上运行，但苦于运行环境不匹配而没有顺利进行下去。  

  
![](https://p.sda1.dev/18/c51ab0cceeee672933fafe17b43e979a/4.png)

## 1.10 使用训练好的模型尝试翻译一句日语句子
首先，我们创建函数来翻译一个新句子，包括获取日语句子、标记化、转换为张量、推理，然后将结果解码回一个中文句子。

In [21]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    # 将输入数据移动到设备上
    src = src.to(device)
    src_mask = src_mask.to(device)
    
    # 编码源语言句子
    memory = model.encode(src, src_mask)
    
    # 初始化目标语言句子
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    
    # 生成目标语言句子
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
          break
    return ys

def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
    # 将模型设置为评估模式
    model.eval()
    
    # 对源语言句子进行标记化
    tokens = [BOS_IDX] + [src_vocab.stoi[tok] for tok in src_tokenizer.encode(src, out_type=str)]+ [EOS_IDX]
    num_tokens = len(tokens)
    
    # 将源语言句子转换为张量
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    
    # 使用贪婪解码生成目标语言句子
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    
    # 将生成的目标语言句子转换为文本
    return " ".join([tgt_vocab.itos[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")


然后，我们只需调用 translate 函数并传入所需的参数


In [22]:
translate(transformer, "HSコード 8515 はんだ付け用、ろう付け用又は溶接用の機器(電気式(電気加熱ガス式を含む。)", ja_vocab, en_vocab, ja_tokenizer)


' ▁H S ▁ 代 码 ▁85 15 ▁ 是 焊 接 、 焊 接 或 焊 接 设 备 ( 包 括 电 气 加 热 气 体 ) 。 '

In [23]:
trainen.pop(5)

'美国 设施: 停车场, 24小时前台, 健身中心, 报纸, 露台, 禁烟客房, 干洗, 无障碍设施, 免费停车, 上网服务, 电梯, 快速办理入住/退房手续, 保险箱, 暖气, 传真/复印, 行李寄存, 无线网络, 免费无线网络连接, 酒店各处禁烟, 空调, 阳光露台, 自动售货机(饮品), 自动售货机(零食), 每日清洁服务, 内部停车场, 私人停车场, WiFi(覆盖酒店各处), 停车库, 无障碍停车场, 简短描述Gateway Hotel Santa Monica酒店距离海滩2英里(3.2公里),提供24小时健身房。每间客房均提供免费WiFi,客人可以使用酒店的免费地下停车场。'

In [25]:
trainja.pop(5)

'パリ 施設・設備: 駐車場, バー, 24時間対応フロント, 新聞, 禁煙ルーム, 空港シャトル, ランドリー, 朝食ルームサービス, インターネット, エレベーター, セーフティボックス, ツアーデスク, 暖房, FAX / コピー, Wi-Fi, 無料Wi-Fi, 全館禁煙, エアコン, コンシェルジュサービス, 空港シャトル(有料), 客室清掃サービス(毎日), Wi-Fi(館内全域), 短い説明Le Notre Dame Saint Michelはパリ中心部のセーヌ河岸にある魅力的なブティックホテルです。'

## 1.11 保存词汇对象和训练好的模型
最后，在训练完成后，我们将首先使用Pickle保存词汇对象（en_vocab 和 ja_vocab）。

In [26]:
import pickle
# 打开一个文件，用于存储数据
file = open('en_vocab.pkl', 'wb')
# 将信息存储到文件中
pickle.dump(en_vocab, file)
file.close()
file = open('ja_vocab.pkl', 'wb')
pickle.dump(ja_vocab, file)
file.close()

最后，我们还可以使用 PyTorch 的保存和加载函数来保存模型以供以后使用。一般来说，根据我们以后想要如何使用模型，有两种保存模型的方式。第一种方式是仅用于推理，我们可以稍后加载模型并用它来从日语翻译成中文。


In [27]:
# 保存用于推理的模型
torch.save(transformer.state_dict(), 'inference_model')

The second one is for inference too, but also for when we want to load the model later, and want to resume the training.



In [28]:
# 保存模型和检查点以便以后恢复训练
torch.save({
  'epoch': NUM_EPOCHS,
  'model_state_dict': transformer.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),
  'loss': train_loss,
  }, 'model_checkpoint.tar')

# 1.12 总结
在包括希冀和kaggle等云平台上进行环境配置遇到了较多困难，最后决定在本地配置cuda用GPU进行训练，需要注意的是根据自己电脑的配置来安装对应版本的cuda及cudnn，配置环境变量，cuda版本和pytorch版本的匹配。